<a href="https://colab.research.google.com/github/shlear/MLDM-2022/blob/main/exam_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np                                   # utilities for numeric operations
import pandas as pd                                  # tabular data processing
from sklearn.model_selection import train_test_split # splitting dataset into random train and test subsets.
from scipy.stats import pearsonr                     # Pearson correlation coefficient
import lightgbm as lgb                               # Light GBM

In [1]:
from google.colab import files # Function for uploading files
files.upload()                 # Upload kaggle.json file

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"egorbevz","key":"60602a5f249baeadc5d7ef09fdbe23af"}'}

In [ ]:
! pip install -q kaggle                                                                  # Install kaggle API
! cp kaggle.json ~/.kaggle/                                                              # Put the kaggle.json into the ~/.kaggle directory
! chmod 600 ~/.kaggle/kaggle.json                                                        # Make the kaggle.json executable
! kaggle datasets download robikscube/ubiquant-parquet -f train_low_mem.parquet --path . # Download train_low_mem.parquet.zip
! unzip train_low_mem.parquet.zip                                                        # Unzip train_low_mem.parquet.zip

 99% 3.05G/3.07G [00:27<00:00, 67.9MB/s]
100% 3.07G/3.07G [00:27<00:00, 119MB/s] 
Archive:  train_low_mem.parquet.zip
  inflating: train_low_mem.parquet   




```
# Выбран кодовый формат
```

# Prepare Data

### Load

In [ ]:
# train_low_mem.parquet is a huge file, so we are going to use:
# * N features
# * M data points for model training
# * the lest M points for model evaluation

N = 100
M = 1000000
features = [f'f_{i}' for i in range(N)]                    # Use columns f_0, f_1, ..., f_N-1 as features

X_train_reduced = pd.read_parquet(
    'train_low_mem.parquet',
    columns=features+['target']
    )[-2*M:-M]                                             # Load train data (M lines before the last M lines), load only those columns listed in features and the `target`
y_train_reduced = X_train_reduced['target']                # Set the column `target` as a target array for model fitting
X_train_reduced = X_train_reduced.drop(columns=['target']) # Drop the column `target` from X_train_reduced, so X_train_reduced contains only features for model fitting
display(X_train_reduced, y_train_reduced)                  # Display the features table (X_train_reduced) and the target array (y_train_reduced)


,f_0,f_1,f_2,f_3,f_4,f_5,f_6,f_7,f_8,f_9,...,f_90,f_91,f_92,f_93,f_94,f_95,f_96,f_97,f_98,f_99
1141410,1.093324,1.301436,-1.242253,5.828377,-0.384234,1.447711,1.519053,0.998088,0.328072,0.781790,...,-0.487156,2.013887,2.145116,-0.565078,0.954476,-0.104428,-0.493579,0.562440,1.109892,-0.343760
1141411,0.104047,-0.113154,0.719374,-0.210036,-0.433466,0.141140,0.893335,0.340964,0.328072,1.189662,...,-0.735104,-0.309837,-0.278825,0.700176,-0.608110,0.484129,-0.824128,-0.789265,0.328594,-0.343760
1141412,1.436316,-0.517323,0.953603,-0.214181,-0.541468,0.125799,0.958002,0.950252,0.328072,-0.748747,...,-0.666868,0.059887,1.580489,0.716956,1.041286,1.185396,-1.433365,-0.992975,-0.025335,1.855881
1141413,-0.678037,0.291014,0.467504,-0.241889,-0.366275,0.193799,-0.179444,-0.197057,0.328072,-1.003611,...,-0.060639,-1.202124,-0.339710,0.441695,0.334402,0.209693,-0.258891,-0.676576,-0.028874,-0.343760
1141414,0.142369,1.301436,-1.456780,1.780014,-0.151572,0.541044,-0.859400,-0.197057,0.328072,-0.360194,...,1.091521,0.828786,1.394190,0.454494,1.983799,-0.447326,1.509864,-0.145400,0.996635,-0.343760
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2141405,1.327377,-0.552583,-0.886323,0.014043,0.125454,1.677687,-1.381808,-1.099508,0.526042,-1.925724,...,-0.479163,-0.026399,-0.805029,0.138502,2.100753,-0.970065,-2.638478,0.682677,0.194035,-0.382265
2141406,0.236709,-2.310760,-0.465879,0.433303,-0.253564,-1.375251,0.839487,1.075578,0.189406,0.142188,...,1.705065,1.222087,-0.243805,-0.624759,-0.029950,0.294779,2.685435,-0.146490,0.276808,0.479031
2141407,0.464893,0.201072,-2.056188,-0.535923,0.207322,-0.212354,1.799059,-0.312653,0.817783,-0.697212,...,0.298559,0.316116,-0.647126,-1.071017,0.671536,-0.514084,0.871859,0.663646,2.068613,-0.389321
2141408,0.290392,0.501166,2.708688,-0.410148,-0.400013,-0.949561,-1.068481,1.539986,0.206195,-0.769957,...,-0.497587,-0.264609,-0.256282,-1.236324,-2.198807,2.692805,-1.020720,-1.041804,-1.319790,1.017152


1141410   -1.705758
1141411   -0.260084
1141412    0.013137
1141413    0.841180
1141414   -0.086323
             ...   
2141405    1.304217
2141406   -0.376348
2141407   -0.854828
2141408   -0.336876
2141409    0.775711
Name: target, Length: 1000000, dtype: float32

[Текст ссылки](https://)# Create Model - LGBM

In [ ]:
model_lgb = lgb.LGBMRegressor()                 # Create an instance of Light GBM regressor
model_lgb.fit(X_train_reduced, y_train_reduced) # Fit the Light GBM regressor

# Light GBM - Gradient Boosting on Decision Trees Algorithm -> State-of-the-Art for tabular data

LGBMRegressor()

# Inference

In [ ]:
X_test_reduced = pd.read_parquet(
    'train_low_mem.parquet',
    columns=features+['target']
    )[-M:]                                               # Load test data (the last M lines)
y_test_reduced = X_test_reduced['target']                # Set the column `target` as a target array for assessing model performance
X_test_reduced = X_test_reduced.drop(columns=['target']) # Drop the column `target` from X_test_reduced
display(X_test_reduced, y_test_reduced)                  # Display the features table (X_test_reduced) and the target array (y_test_reduced)


,f_0,f_1,f_2,f_3,f_4,f_5,f_6,f_7,f_8,f_9,...,f_90,f_91,f_92,f_93,f_94,f_95,f_96,f_97,f_98,f_99
2141410,-0.318628,1.878033,-0.254857,-0.034804,-0.134778,-0.646280,-0.099820,-0.338968,0.117094,-0.117497,...,-0.889317,0.050175,-1.007735,1.266060,0.307827,0.431842,-0.566316,-0.373920,-1.237720,0.987137
2141411,-0.793941,-0.635127,0.737949,-0.655404,-0.208031,-0.811264,0.411129,-0.607927,0.131232,1.100606,...,1.706120,0.055868,0.038690,-1.380178,-2.326279,0.699331,0.191068,-0.685449,0.104734,0.101148
2141412,1.056986,-1.240384,1.003570,-0.624688,-0.300291,0.699789,0.107452,0.865332,0.098041,2.373290,...,0.504580,-0.473205,-0.768706,-0.169144,-0.899174,0.772994,0.636828,-0.654049,-1.486701,-0.595339
2141413,-1.622636,1.302770,-0.694586,-0.403037,-0.176304,1.265605,1.415097,0.083107,0.071317,0.955035,...,2.575595,-0.995329,0.874402,-0.722395,-1.938771,-0.636336,0.207978,-0.181923,-0.589655,-0.368053
2141414,0.040462,-0.075609,-0.606644,0.167259,-0.098879,-0.085549,1.016559,1.935746,0.083219,0.213701,...,1.303252,-0.164346,-0.504350,-1.991226,0.707544,-0.278499,0.905299,-0.133844,-0.181467,-0.223240
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3141405,0.093530,-0.720275,-0.345497,-0.438781,-0.166972,-0.437182,1.475746,1.284423,0.056425,-1.433681,...,-0.265811,-1.018476,0.453945,0.395551,0.318854,0.725995,-0.596743,-0.591497,-0.830558,-0.217642
3141406,-1.344935,-0.199987,-0.107702,-0.454677,-0.221914,-0.141174,-1.498235,1.373834,0.056425,-1.211572,...,-0.707159,0.036467,0.002782,0.424902,0.605383,1.704695,-0.625599,-1.110052,-0.946176,-0.217642
3141407,0.979489,-1.110491,1.006980,-0.467307,-0.159549,1.355671,0.150812,-0.088923,0.056425,0.996380,...,-0.265811,0.483454,-0.537039,0.363976,0.318854,0.132587,-1.064116,-0.383061,1.458104,-0.217642
3141408,-2.565332,0.320301,0.076600,1.380182,-0.155366,-0.689000,0.381069,-1.324759,0.056425,-1.111730,...,-0.265811,0.050828,0.104241,0.448094,0.605383,0.269845,-0.303289,-0.229851,0.026150,-0.217642


2141410    0.678174
2141411    0.059027
2141412    0.274764
2141413    0.159570
2141414   -1.265211
             ...   
3141405    0.033600
3141406   -0.223264
3141407   -0.559415
3141408    0.009599
3141409    1.212112
Name: target, Length: 1000000, dtype: float32

In [ ]:
prediction_train = model_lgb.predict(X_train_reduced) # Predict values for train set using the fitted model
prediction_test = model_lgb.predict(X_test_reduced)   # Predict values for test set using the fitted model 

# Evaluate

In [ ]:
print(
    'Train Pearson r:', pearsonr(y_train_reduced, prediction_train)[0], '\n',
    'Test Pearson r:', pearsonr(y_test_reduced, prediction_test)[0],
) # Display Pearson correlation coefficient for train and test sets

Train Pearson r: 0.2368148605891157 
 Test Pearson r: 0.0914568869131924
